# Projeto Final
## Matheus Freitas Martins

In [2]:
import requests
import pandas as pd

In [3]:
# URL da API do Mercado Livre para buscar produtos
url = "https://api.mercadolibre.com/sites/MLB/search?q=produto"

In [4]:
# Realiza a requisição para a API
response = requests.get(url)

# Converte a resposta para JSON
data = response.json()

# Extrai os resultados
results = data['results']


In [5]:
df = pd.DataFrame(results)
df.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,...,attributes,installments,winner_item_id,catalog_listing,discounts,promotions,inventory_id,differential_pricing,variation_filters,variations_data
0,MLB2669984650,Fraldas Para Adultos Plenitud Descartável Pro...,new,707487-MLU48309839238_112021,MLB18719562,gold_special,https://www.mercadolivre.com.br/fraldas-para-a...,buy_it_now,MLB,MLB264199,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 12, 'amount': 5.29, 'rate': 16.64...",None,True,None,[],SCKP57104,NaN,NaN,NaN
1,MLB3497694692,Widi Care Modelando A Juba Geleia Seladora Cac...,new,848806-MLB49714682978_042022,MLB22621799,gold_special,https://www.mercadolivre.com.br/widi-care-mode...,buy_it_now,MLB,MLB32130,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 12, 'amount': 3.84, 'rate': 16.34...",None,True,None,[],QWMD15303,NaN,NaN,NaN
2,MLB2758609100,Tend Skin 118ml - Loção Para Foliculite E Pelo...,new,749826-MLB47043569114_082021,MLB19510519,gold_pro,https://www.mercadolivre.com.br/tend-skin-118m...,buy_it_now,MLB,MLB1262,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 5, 'amount': 31.8, 'rate': 0, 'cu...",None,True,None,[],GTVT00302,{'id': 35713361},NaN,NaN
3,MLB3269837602,Loção hidratante Com Ácido Hialurônico 473ml C...,new,712555-MLA53367010053_012023,MLB21361292,gold_special,https://www.mercadolivre.com.br/locohidratante...,buy_it_now,MLB,MLB1262,...,"[{'id': 'UNITS_PER_PACKAGE', 'name': 'Unidades...","{'quantity': 12, 'amount': 8.75, 'rate': 16.64...",None,True,None,[],LNES53233,NaN,NaN,NaN
4,MLB1722033040,Kit 12 Galões 5 Litros Vazio Pead Virgem Produ...,new,902234-MLB44120127005_112020,None,gold_special,https://produto.mercadolivre.com.br/MLB-172203...,buy_it_now,MLB,MLB244658,...,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...","{'quantity': 12, 'amount': 7.2, 'rate': 16.64,...",None,NaN,None,[],None,NaN,[COLOR],{'68680207831': {'thumbnail': 'http://http2.ml...
